In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-muted')
from matplotlib import cm
import geopandas as gpd

from pyproj import Transformer
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import multiprocessing
from multiprocessing import Pool
from functools import partial
import numpy.random as npr
import matplotlib.dates as mdates
from itertools import product
import copy
import pickle
from tqdm import tqdm

import nomad.io.base as loader
import nomad.city_gen as cg
from nomad.city_gen import City, Building
import nomad.traj_gen as tg
from nomad.traj_gen import Agent, Population
import nomad.stop_detection.ta_dbscan as DBSCAN
import nomad.stop_detection.lachesis as Lachesis
from nomad.generation.sparsity import gen_params_target_q

In [2]:
city_geojson = gpd.read_file('garden_city.geojson')
city = cg.load('garden-city.pkl')

### Generate N agents

The following code maps our Garden City coordinates to a location in the Atlantic Ocean (Atlantis?).

In [3]:
def garden_city_to_lat_long(agent, sparse_traj=True, full_traj=False, diaries=True):
    def project_city_blocks_to_web_mercator(df):
        """Convert (x, y) from 15m block units to Web Mercator meters via affine shift and projection."""
        transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
        df['x'] = 15 * df['x'] - 4265699
        df['y'] = 15 * df['y'] + 4392976
        if 'ha' in df:
            df['ha'] = 15 * df['ha']
        df['longitude'], df['latitude'] = transformer.transform(df['x'].values, df['y'].values)
        df['date'] = df['datetime'].dt.date
        return df

    def finalize(df):
        front = ['identifier', 'timestamp', 'longitude', 'latitude', 'x', 'y', 'date']
        cols = [col for col in front if col in df] + [col for col in df.columns if col not in front]
        return df[cols].rename(columns={'identifier': 'uid', 'timestamp': 'timestamp'}).reset_index(drop=True)

    if sparse_traj:
        agent.sparse_traj = finalize(project_city_blocks_to_web_mercator(agent.sparse_traj))
    if full_traj:
        agent.trajectory = finalize(project_city_blocks_to_web_mercator(agent.trajectory))
        
    if diaries:
        diary = agent.diary.copy()
        xs = []
        ys = []
        for loc in diary["location"]:
            if loc is None:
                xs.append(None)
                ys.append(None)
            else:
                pt = agent.city.buildings[loc].geometry.centroid
                xs.append(pt.x)
                ys.append(pt.y)
        diary["x"] = xs
        diary["y"] = ys
        agent.diary = finalize(project_city_blocks_to_web_mercator(diary))

Initiate $N$ empty agents. 

## Simple trajectory generation

For simple trajectory generation tasks that don't require too much computation power and can be done on a personal laptop, the following code generates a trajectory for each agent and saves it to a csv.

In [ ]:
def generate_agent_trajectory(agent_id, agent, seed):

    beta_params = gen_params_target_q(q_range=(0.4, 0.85), seed=seed)

    agent.generate_trajectory(
        datetime = "2024-01-01T08:00 -04:00",
        end_time = pd.Timestamp('2024-01-21T08:30:00 -04:00'),
        seed=1,
        dt=0.25)
    print('finished generating trajectory')
    agent.sample_trajectory(
        **beta_params,
        seed=seed,
        ha=13/15, # <<<<<<
        replace_sparse_traj=True)
    
    garden_city_to_lat_long(agent,
                            sparse_traj=True,
                            full_traj=False)
    agent.reset_trajectory(trajectory = True, sparse = False, diary = False)
    
    return agent_id, copy.deepcopy(agent)

In [5]:
population = Population(city)
population.generate_agents(N=2, seed=2, name_count=3)

In [6]:
%%time 
# time for 2 weeks of data at dt = 0.25
agent_1 = list(population.roster.values())[1]
generate_agent_trajectory(agent_1, 10, 6)
agent_1.sparse_traj.head()

finished generating trajectory
CPU times: total: 8.27 s
Wall time: 8.31 s


,uid,timestamp,longitude,latitude,x,y,date,datetime,ha
0,admiring_adoring_banach,1704116813,-38.317481,36.670075,-4.265483e+06,4.393219e+06,2024-01-01,2024-01-01 09:46:53-04:00,12.137903
1,admiring_adoring_banach,1704117085,-38.317457,36.670111,-4.265480e+06,4.393224e+06,2024-01-01,2024-01-01 09:51:25-04:00,8.972810
2,admiring_adoring_banach,1704117157,-38.317496,36.670122,-4.265484e+06,4.393225e+06,2024-01-01,2024-01-01 09:52:37-04:00,8.323459
3,admiring_adoring_banach,1704117285,-38.317533,36.670070,-4.265488e+06,4.393218e+06,2024-01-01,2024-01-01 09:54:45-04:00,14.388030
4,admiring_adoring_banach,1704117646,-38.317426,36.670088,-4.265476e+06,4.393221e+06,2024-01-01,2024-01-01 10:00:46-04:00,12.852338


In [14]:
population = Population(city)
population.generate_agents(N=2, seed=2, name_count=3)

In [15]:
%%time 
# time for 2 weeks of data at dt = 0.25
agent_1 = list(population.roster.values())[1]
generate_agent_trajectory(agent_1, 10, 6)
agent_1.sparse_traj.head()

finished generating trajectory
CPU times: total: 7.09 s
Wall time: 7.08 s


,uid,timestamp,longitude,latitude,x,y,date,datetime,ha
0,admiring_adoring_banach,1704116813,-38.318866,36.669763,-4.265637e+06,4.393175e+06,2024-01-01,2024-01-01 09:46:53-04:00,12.014135
1,admiring_adoring_banach,1704117085,-38.318863,36.669797,-4.265636e+06,4.393180e+06,2024-01-01,2024-01-01 09:51:25-04:00,8.947531
2,admiring_adoring_banach,1704117157,-38.318856,36.669787,-4.265636e+06,4.393179e+06,2024-01-01,2024-01-01 09:52:37-04:00,8.315352
3,admiring_adoring_banach,1704117285,-38.318824,36.669771,-4.265632e+06,4.393176e+06,2024-01-01,2024-01-01 09:54:45-04:00,14.181900
4,admiring_adoring_banach,1704117646,-38.318787,36.669754,-4.265628e+06,4.393174e+06,2024-01-01,2024-01-01 10:00:46-04:00,12.703410


In [ ]:
def generate_trajectory_data(agent_id, agent, seed):
    beta_params = gen_params_target_q(q_range=(0.4, 0.85), seed=seed)
    ha_sample = npr.uniform(11.5/15, 14.5/15)

    agent.reset_trajectory()
    agent.generate_trajectory(
        datetime = "2024-01-01T08:00 -04:00",
        end_time = pd.Timestamp('2024-01-21T08:00:00 -04:00'),
        seed=1,
        dt=1)

    agent.sample_trajectory(
        **beta_params,
        seed=seed,
        ha=ha_sample,
        replace_sparse_traj=True)

    garden_city_to_lat_long(agent, sparse_traj=True, full_traj=False)
    agent.reset_trajectory(trajectory = True, sparse = False, diary = False)
    return None

# Generate trajectories with progress bar
N = 200  
population = Population(city)
population.generate_agents(N=N, seed=1, name_count=3)

for agent_id, agent in tqdm(population.roster.items(), desc="Generating trajectories"):
    generate_trajectory_data(agent_id, agent, seed=1)

<<<<<<< local


Generating trajectories: 123it [04:05,  2.00s/it]

>>>>>>> remote


<<<<<<< local <modified: >


Generating trajectories: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [06:35<00:00,  1.98s/it]


>>>>>>> remote <removed>


In [ ]:
traj_cols = {
    "user_id": "uid",
    "timestamp": "timestamp",
    "latitude": "latitude",
    "longitude": "longitude",
    "x": "x",
    "y": "y",
    "duration": "duration",
    "datetime": "datetime"}
# Save only sparse trajectories and diaries
population.save_pop(
    sparse_path="output/sparse_traj/",
    diaries_path="output/diaries/",
    partition_cols={"sparse_traj": ["date"]},
    traj_cols=traj_cols
)

In [ ]:
sparse_df = loader.from_file("output/sparse_traj/", format="parquet", traj_cols=traj_cols,
                      parse_dates=True)
diaries_df = loader.from_file("output/diaries/", format="parquet", traj_cols=traj_cols,
                       parse_dates=True)

In [ ]:
diaries_df.head()

,uid,timestamp,longitude,latitude,x,y,date,datetime,duration,location,tz_offset
0,unruffled_dijkstra,1704106860,-38.318415,36.669407,-4265586.5,4393126.0,2024-01-01,2024-01-01 07:01:00,60,h-x8-y10,-14400
1,unruffled_dijkstra,1704110460,NaN,NaN,NaN,NaN,2024-01-01,2024-01-01 08:01:00,4,None,-14400
2,unruffled_dijkstra,1704110700,-38.317674,36.670163,-4265504.0,4393231.0,2024-01-01,2024-01-01 08:05:00,236,w-x13-y18,-14400
3,unruffled_dijkstra,1704124860,NaN,NaN,NaN,NaN,2024-01-01,2024-01-01 12:01:00,4,None,-14400
4,unruffled_dijkstra,1704125100,-38.316731,36.670542,-4265399.0,4393283.5,2024-01-01,2024-01-01 12:05:00,11,r-x20-y21,-14400


In [ ]:
sparse_df.head()

<<<<<<< local <modified: text/html, text/plain>


,uid,timestamp,longitude,latitude,x,y,datetime,ha,date,tz_offset
0,unruffled_dijkstra,1704112023,-38.317844,36.670325,-4.265523e+06,4.393253e+06,2024-01-01 08:27:03,16.329762,2024-01-01,-14400
1,unruffled_dijkstra,1704112203,-38.317545,36.670340,-4.265490e+06,4.393255e+06,2024-01-01 08:30:03,9.474195,2024-01-01,-14400
2,unruffled_dijkstra,1704112273,-38.317589,36.670253,-4.265494e+06,4.393243e+06,2024-01-01 08:31:13,12.021023,2024-01-01,-14400
3,unruffled_dijkstra,1704112372,-38.317579,36.670279,-4.265493e+06,4.393247e+06,2024-01-01 08:32:52,8.604520,2024-01-01,-14400
4,unruffled_dijkstra,1704112663,-38.317738,36.670173,-4.265511e+06,4.393232e+06,2024-01-01 08:37:43,13.937659,2024-01-01,-14400


>>>>>>> remote <removed>


## Parallelized Trajectory Generation

For larger trajectory generation tasks that require a lot of compute power, we can parallelize the trajectory generation using the following code. We generate ground-truth trajectories in agent-month "chunks", sparsify each chunk, then reset the ground-truth trajectory field to lessen the memory usage. 

In [ ]:
# Using parallel processing (e.g., using a cluster)
%%time

def generate_trajectory_data(agent_id, agent, seed):
    
    beta_duration = npr.uniform(15, 180)
    beta_start = max(npr.uniform(60, 1200), beta_duration*3)
    beta_ping = npr.uniform(1.5, 30)
    
    param = (beta_start, beta_duration, beta_ping)
    
    for month in range(1,13):
        days = calendar.monthrange(2024, month)[1]
        population_n.generate_trajectory(agent, 
                                         T=datetime(2024, month, days, hour=23, minute=59), 
                                         seed=seed)
    
        agent.sample_traj_hier_nhpp(*param, 
                                    seed=seed,
                                    reset_traj=True)
    
    garden_city_to_lat_long(agent,
                            sparse_traj=True,
                            full_traj=False)
    
    return agent_id, copy.deepcopy(agent)

manager = multiprocessing.Manager()
shared_roster = manager.dict(population_n.roster)

start = 6001  # 12001  # can modify
end = 12001   # 18001  # can modify
roster = dict(population_n.roster)
batch = islice(roster.items(), start, end)

with ProcessPoolExecutor() as executor:
    with tqdm(total=(end-start), desc="Processing agents") as pbar:
        futures = [
            executor.submit(generate_trajectory_data, agent_id, agent, i+15000)
            for i, (agent_id, agent) in enumerate(batch, start=start)
        ]
        results = []
        for future in futures:
            results.append(future.result())
            pbar.update(1)

for agent_id, agent in results:
    population_n.roster[agent_id] = agent

This code saves the generated trajectories in a parquet file, using the date as the partition column.

In [ ]:
partition_cols = {
    'sparse_traj': ['date'],
    'diaries': ['identifier']
}

roster = dict(islice(population_n.roster.items(), start, end))

population_n.save_pop(bucket="synthetic-raw-data",
                      prefix=f"agents-{start+15000}-{end+15000-1}/",
                      save_full_traj=False,
                      save_sparse_traj=True,
                      save_homes=True,
                      save_diaries=True,
                      partition_cols=partition_cols,
                      roster=roster)

In [ ]:
# Load the Parquet files
s3_path = "s3://synthetic-raw-data/agents-1-1001/sparse_trajectories.parquet/"
df1 = pd.read_parquet(s3_path)
s3_path = "s3://synthetic-raw-data/agents-1001-2000/sparse_trajectories.parquet/"
df2 = pd.read_parquet(s3_path)